In [3]:
import numpy as np 
import pandas as pd 
from sklearn.metrics.pairwise import linear_kernel
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import re
import string

In [5]:
trainning_df = pd.read_excel('/kaggle/input/gov-tn/Classeur1.xltx')

In [6]:
trainning_df.head()

,website,paragraph,question
0,http://www.finances.gov.tn/fr/,Le décret n° 75-316 du 30 mai 1975 fixe les at...,C'est quoi le ministère de la finance et quell...
1,NaN,Création :\nLe CNF est crée par l’article 4 du...,C'est quoi le sonseil national de la fiscalité...
2,NaN,Mme Sihem Boughdiri Nemsia nommée par le prési...,Qui est la minitstre des finances
3,NaN,Le conseil supérieur de comptabilité a été cré...,C'est quoi le conseil de comptabilité ?
4,NaN,Le Conseil National de la Comptabilité est com...,Quelle est la composition du conseil de la com...


In [7]:
trainning_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   website    5 non-null      object
 1   paragraph  227 non-null    object
 2   question   227 non-null    object
dtypes: object(3)
memory usage: 5.4+ KB


In [8]:
trainning_df.isnull().sum()

website      222
paragraph      0
question       0
dtype: int64

In [9]:
trainning_question_ls = list(trainning_df['question'])
trainning_answer_ls = list(trainning_df['paragraph'])


In [10]:
print(len(trainning_answer_ls))
print(len(trainning_question_ls))

227
227


answerdata augmentation 

In [11]:
ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(trainning_question_ls).reshape(-1, 1), np.array(trainning_answer_ls).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['question', 'answer']);

In [12]:
#train_os.info()

In [13]:
#train_os.head()

In [14]:
# Eliminer les stop words: les mots très fréquents en anglais
nltk.download('stopwords')
final_stopwords_list = stopwords.words('english') + stopwords.words('french')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if not (word.lower() in final_stopwords_list)])

In [16]:
#Eliminer les ponctuations
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [17]:
#Eliminer les nombres
def cleaning_numbers(data):
    return re.sub('[0-9]+','', data)

In [18]:
def strip_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\t', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    banned_list= string.punctuation +'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

In [19]:
keywords=list()
for item in train_os['answer'] :
    item_num=cleaning_numbers(item)
    item_semi_clean=cleaning_punctuations(item_num)
    item_semi=strip_entities(item_semi_clean)
    item_clean=cleaning_stopwords(item_semi)
    # Initialize the TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words=final_stopwords_list)

    # Fit the TfidfVectorizer to the text
    tfidf_vectorizer.fit([item_clean])

    # Get the feature names and their corresponding scores
    feature_names = tfidf_vectorizer.get_feature_names_out() 
    feature_scores = tfidf_vectorizer.idf_

    # Combine the feature names and their corresponding scores into a dictionary
    feature_dict = dict(zip(feature_names, feature_scores))

    # Sort the dictionary by the feature scores in descending order
    sorted_features = sorted(feature_dict.items(), key=lambda x: x[1], reverse=True)
    keywords.append(sorted_features[0][0]+','+sorted_features[1][0]+','+sorted_features[2][0])

In [20]:
print(keywords)

['accorde,accords,administratif', 'agriculture,article,artisanat', 'administration,afférentes,ailleurs', 'article,attribué,composition', 'administratif,banque,bvmt', 'auprès,auxiliaires,besoin', 'amélioration,application,avis', 'accordé,accounting,adoptée', 'accorder,achat,activité', 'adéquates,ainsi,appréciation', 'approbation,arrêtés,cadre', 'affaires,affectés,archives', 'affaires,agricole,ainsi', 'agence,agricole,alcools', 'accès,administratif,ar', 'acquis,administration,admis', 'ainsi,aligner,amélioration', 'afférentes,ainsi,application', 'administration,administrative,adopte', 'ajouté,août,approbation', 'ar,cette,commission', 'avis,cnncp,collectivités', 'accès,action,adapter', 'avis,cnncp,comptes', 'année,besoins,budget', 'ajoutée,assurances,certains', 'abattements,absence,accordés', 'acomptes,actions,actionsau', 'acceptés,accompagnement,achats', 'ad,alcool,alcoolisées', 'accompagnée,accordés,acheter', 'agricoles,assurance,autres', 'activité,affaires,aménagement', 'ajoutée,août,av

In [21]:
df = pd.DataFrame(columns=['question','answer','Keywords'])
df['question'] =['question: '+item for item in train_os['question']]
df['answer'] = ['answer: '+item for item in train_os['answer']]
df['Keywords']=[' Keywords : '+ keyword for keyword in keywords]

In [22]:
df_final = pd.DataFrame(columns=['question','answer'])
df_final['question'] = trainning_question_ls
df_final['answer']=df['answer']+ df['Keywords']

In [23]:
df_final['answer'][0]

'answer: Le décret n° 75-316 du 30 mai 1975 fixe les attributions du Ministère des Finances qui a pour mission essentielle l’élaboration et la mise en œuvre de la politique de l’Etat en matière financière, monétaire et fiscale. Dans ce cadre, il participe à la conception des objectifs de la politique économique et à la définition des moyens de réalisation appropriés. \n\nLe Ministère des Finances a aussi pour mission :\n\nD’élaborer les projets des lois de finances et de veiller, conformément à la législation et à la réglementation en vigueur à l’exécution du budget de l’Etat, des budgets annexes, des budgets des établissements publics à caractère administratif, ainsi que des fonds spéciaux du trésor. A ce titre, il est consulté sur toutes les questions ayant une incidence budgétaire et notamment celles afférentes aux rémunérations publiques ;\nDe préparer les projets de textes à caractère fiscal et douanier et assure l’exécution de la législation dans ce domaine ;\nDe préparer les pro

In [24]:
df.head()

,question,answer,Keywords
0,question: C'est quoi le ministère de la financ...,answer: Le décret n° 75-316 du 30 mai 1975 fix...,"Keywords : accorde,accords,administratif"
1,question: C'est quoi le sonseil national de la...,answer: Création :\nLe CNF est crée par l’arti...,"Keywords : agriculture,article,artisanat"
2,question: Qui est la minitstre des finances,answer: Mme Sihem Boughdiri Nemsia nommée par ...,"Keywords : administration,afférentes,ailleurs"
3,question: C'est quoi le conseil de comptabilité ?,answer: Le conseil supérieur de comptabilité a...,"Keywords : article,attribué,composition"
4,question: Quelle est la composition du conseil...,answer: Le Conseil National de la Comptabilité...,"Keywords : administratif,banque,bvmt"


In [25]:
df_final.head()

,question,answer
0,C'est quoi le ministère de la finance et quell...,answer: Le décret n° 75-316 du 30 mai 1975 fix...
1,C'est quoi le sonseil national de la fiscalité...,answer: Création :\nLe CNF est crée par l’arti...
2,Qui est la minitstre des finances,answer: Mme Sihem Boughdiri Nemsia nommée par ...
3,C'est quoi le conseil de comptabilité ?,answer: Le conseil supérieur de comptabilité a...
4,Quelle est la composition du conseil de la com...,answer: Le Conseil National de la Comptabilité...


**MODEL**

In [26]:
!pip install transformers==4.14.1 -q
!pip install bitsandbytes-cuda111==0.26.0 -q
!pip install datasets==1.16.1 -q

  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib.linux-x86_64-cpython-310/tokenizers
      creating build/lib.linux-x86_64-cpython-310/tokenizers/models
      copying py_src/tokenizers/models/__init__.py -> build/lib.linux-x86_64-cpython-310/tokenizers/models
      creating build/lib.linux-x86_64-cpython-310/tokenizers/decoders
      copying py_src/tokenizers/decoders/__init__.py -> build/lib.linux-x86_64-cpython-310/tokenizers/decoders
      creating build/lib.linux-x86_64-cpython-310/tokenizers/normalizers
      copying py_src/tokenizers/normalizers/__init__.py -> build/lib.linux-x86_64-cpython-310/tokenizers/normalizer

In [27]:
from sklearn.model_selection import train_test_split
import transformers
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd
from tqdm.auto import tqdm

In [28]:
class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias
 
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output += self.adapter(input)
        return output
 
    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"
 
 
class DequantizeAndLinear(torch.autograd.Function): 
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias).clone()
 
    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias
 
 
class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
 
    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output 
 
    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"
 
 
def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)
 
    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)
 
 
def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr(
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

In [29]:
class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)
        

class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock

In [30]:
class T5ForConditionalGeneration(transformers.models.t5.modeling_t5.T5ForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)

transformers.models.t5.modeling_t5.T5ForConditionalGeneration = T5ForConditionalGeneration

In [31]:
config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [32]:
config.pad_token_id = config.eos_token_id
tokenizer.pad_token = config.pad_token_id

In [33]:
#gpt = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True)
gpt = GPTJForCausalLM.from_pretrained("gustavecortal/fr-boris-8bit", low_cpu_mem_usage=True) 

k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

In [34]:
!gdown --id 1Q1WMjny26VHLKb71iTCHIS5zvdm9c-wz

/bin/bash: gdown: command not found


In [35]:
data=pd.DataFrame()
data['sentence'] = '[Sentence]:'+'en Tunisie,'+train_os['question']+'\n[Paraphrase]:'+'en Tunisie,'+train_os['answer']
data=data['sentence']
train, test = train_test_split(data, test_size=0.05) 
train.to_csv('/kaggle/working/train_pgbp_example.csv', index=False)
test.to_csv('/kaggle/working/test_pgbp_example.csv', index=False)
data.head()

0    [Sentence]:en Tunisie,C'est quoi le ministère ...
1    [Sentence]:en Tunisie,C'est quoi le sonseil na...
2    [Sentence]:en Tunisie,Qui est la minitstre des...
3    [Sentence]:en Tunisie,C'est quoi le conseil de...
4    [Sentence]:en Tunisie,Quelle est la compositio...
Name: sentence, dtype: object

In [36]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': '/kaggle/working/train_pgbp_example.csv',
                                              'test': '/kaggle/working/test_pgbp_example.csv'})

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-619df89e0752d736/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:170: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:170: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/2 [00:00<?, ?it/s]

In [37]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=True, truncation=True, max_length= 128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence"])
tokenized_datasets.set_format("torch")

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [38]:
from torch.utils.data import DataLoader

full_train_dataset = tokenized_datasets["train"]
train_dataloader = DataLoader(full_train_dataset, shuffle=False, batch_size=8)

In [39]:
def add_adapters(model, adapter_dim=4, p = 0.1):
    assert adapter_dim > 0

    for name, module in model.named_modules():
      if isinstance(module, FrozenBNBLinear):
          if "attn" in name or "mlp" in name or "head" in name:
              print("Adding adapter to", name)
              module.adapter = nn.Sequential(
                nn.Linear(module.in_features, adapter_dim, bias=False),
                nn.Dropout(p=p),
                nn.Linear(adapter_dim, module.out_features, bias=False),
            )
              print("Initializing", name)
              nn.init.zeros_(module.adapter[2].weight)

          else:
              print("Not adding adapter to", name)
      elif isinstance(module, FrozenBNBEmbedding):
          print("Adding adapter to", name)
          module.adapter = nn.Sequential(
                nn.Embedding(module.num_embeddings, adapter_dim),
                nn.Dropout(p=p),
                nn.Linear(adapter_dim, module.embedding_dim, bias=False),
            )
          print("Initializing", name)
          nn.init.zeros_(module.adapter[2].weight)

add_adapters(gpt)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpt.to(device)

Adding adapter to transformer.wte
Initializing transformer.wte
Adding adapter to transformer.h.0.attn.k_proj
Initializing transformer.h.0.attn.k_proj
Adding adapter to transformer.h.0.attn.v_proj
Initializing transformer.h.0.attn.v_proj
Adding adapter to transformer.h.0.attn.q_proj
Initializing transformer.h.0.attn.q_proj
Adding adapter to transformer.h.0.attn.out_proj
Initializing transformer.h.0.attn.out_proj
Adding adapter to transformer.h.0.mlp.fc_in
Initializing transformer.h.0.mlp.fc_in
Adding adapter to transformer.h.0.mlp.fc_out
Initializing transformer.h.0.mlp.fc_out
Adding adapter to transformer.h.1.attn.k_proj
Initializing transformer.h.1.attn.k_proj
Adding adapter to transformer.h.1.attn.v_proj
Initializing transformer.h.1.attn.v_proj
Adding adapter to transformer.h.1.attn.q_proj
Initializing transformer.h.1.attn.q_proj
Adding adapter to transformer.h.1.attn.out_proj
Initializing transformer.h.1.attn.out_proj
Adding adapter to transformer.h.1.mlp.fc_in
Initializing transfor

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): FrozenBNBLinear(4096, 50400)
)

In [40]:
!pip install bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 10.5 MB/s eta 0:00:0000:0100:01


In [41]:
from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/lib/x86_64-linux-gnu'), PosixPath('/usr/local/cuda/lib'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)


In [42]:
from bitsandbytes.optim import Adam8bit

gpt.gradient_checkpointing_enable()
optimizer = Adam8bit(gpt.parameters(), lr=1e-5, weight_decay=0.01)

In [43]:
num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)

In [44]:
lr_scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer, int(num_training_steps*0.1), num_training_steps
)

In [45]:
filepath = '/kaggle/working/model.pt'

In [46]:
from tqdm.auto import tqdm

scaler = torch.cuda.amp.GradScaler()
progress_bar = tqdm(range(num_training_steps))
gpt.train()
gpt.gradient_checkpointing_enable()
k = 0

for epoch in range(num_epochs):
    for batch in train_dataloader:

        k = k + 1
        if k % 500 == 0:
          print(k)
          state = {'k' : k, 'epoch': num_epochs, 'lr_scheduler': lr_scheduler.state_dict(), 'state_dict': gpt.state_dict(), 'optimizer': optimizer.state_dict()}
          torch.save(state, filepath)

        batch = {k: v.to(device) for k, v in batch.items()}

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
          out = gpt.forward(**batch,)

          loss = F.cross_entropy(out.logits[:, :-1, :].flatten(0, -2), batch['input_ids'][:, 1:].flatten(),
                                reduction='mean', label_smoothing=0.1)
          
        print(loss)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(gpt.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        lr_scheduler.step()
        progress_bar.update(1)

  0%|          | 0/270 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


tensor(5.0153, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.3284, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.1454, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.7534, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.8018, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.0680, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.4448, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.1306, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.9682, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.0548, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.3508, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.4443, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.6572, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.0568, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.0162, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.8606, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.0689, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.1443, device='cuda:0', grad_fn=<AddBack

set the question !!!! 

In [47]:
# Compute TF-IDF matrix
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(train_os['answer'])

In [48]:
def TopN(tfidf_matrix,question): 
    gpt.eval()
    with torch.no_grad():
         prompt = tokenizer(question, truncation=True, padding=True, max_length=128, return_tensors='pt')
         prompt = {key: value.to(device) for key, value in prompt.items()}
         out = gpt.generate(**prompt, max_length=128, top_k=50, top_p=0.9, temperature=1.0, do_sample=True, repetition_penalty = 1.2, num_beams=1)
         user_input=tokenizer.decode(out[0])
         # Transform user input into TF-IDF features
         user_tfidf = tfidf.transform([user_input])

         # Compute cosine similarity between user input and listings
         cosine_similarities = linear_kernel(user_tfidf, tfidf_matrix).flatten()

        # Get indices of top 3 listings with highest similarity scores
         top_indices = cosine_similarities.argsort()[:-4:-1]

        # Recommend the most suitable listings
         recommended_listings = df.loc[top_indices]
         topN=' '.join(recommended_listings['answer'])
         # Display the recommended listings
         for index, listing in recommended_listings.iterrows():
             print("Choix: {}".format(index))
             print("Answer: {}".format(listing['answer']))
             print("-----------------------")    

    return topN

In [50]:
# Display the recommended listings
#for index, listing in recommended_listings.iterrows():
   # print("Choix: {}".format(index))
   # print("Answer: {}".format(listing['answer']))
    #print("-----------------------")

Choix: 49
Answer: answer: Le cadre légal et règlementaire dans lequel opère la comptabilité publique est très vaste compte tenu de la multitude et de la variété des opérations financières et comptables induites par l'activité des entités publiques. Les textes les plus importants sont :

Textes législatifs :

Code de la comptabilité publique promulgué par la loi n°73-81 du 31 décembre 1973, tel que modifié notamment par la loi des finances pour la gestion 2010;
Loi n°67-53 du 8 décembre 1967 portant loi organique du budget telle que modifiée notamment par la loi n°2004-42 du 13 mai 2004;
Loi 75-35 du 14 mai 1975 portant loi organique du budget des collectivités locales telle que modifiée notamment par la loi organique n°2007-65 du 18 décembre 2007;
Loi 68-8 du 8 mars 1968 portant organisation de la cour des comptes telle que modifiée notamment par la loi organique n°2008-3 du 29 janvier 2008;
Code de la TVA promulgué par la loi N°88-61 du 2 Juin 1988;
Loi N°88-62 du 2 Juin 1988 Portant 

In [49]:
test.values[0].split('[Paraphrase]:')[0].strip()

"[Sentence]:en Tunisie,qu'est ce qu'une banque d'affaire ?"

In [50]:
gpt.eval()
for sentence in test.values:
  st = sentence.split('[Paraphrase]:')[0].strip()
  phrase=str(st)
  with torch.no_grad():
    prompt = tokenizer(st+TopN(tfidf_matrix,phrase), truncation=True, padding=True, max_length=128, return_tensors='pt')
    prompt = {key: value.to(device) for key, value in prompt.items()}
    out = gpt.generate(**prompt, max_length=512, top_k=50, top_p=0.9, temperature=1.0, do_sample=True, repetition_penalty = 1.2, num_beams=1)
    print('\n')
    print(tokenizer.decode(out[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Choix: 219
Answer: answer: Non : En vertu de l’article 44 du code des assurances, l’assurance des résidents et des risques situés en Tunisie ne peut être souscrite qu’auprès d’une entreprise d’assurances Tunisienne et ce à l’exception de l’assurance de la responsabilité civile de l’armateur et du transporteur maritime.
-----------------------
Choix: 402
Answer: answer: Non : En vertu de l’article 44 du code des assurances, l’assurance des résidents et des risques situés en Tunisie ne peut être souscrite qu’auprès d’une entreprise d’assurances Tunisienne et ce à l’exception de l’assurance de la responsabilité civile de l’armateur et du transporteur maritime.
-----------------------
Choix: 106
Answer: answer: Créée le 19 septembre 1958, la Banque Centrale de Tunisie est un établissement public doté de la personnalité civile et de l'autonomie financière. La loi n° 2016-35 du 25 Avril 2016 portant fixation du statut de la  Banque Centrale a attribué à la banque pour objectif principal d’as

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




[Sentence]:en Tunisie,qu'est ce qu'une banque d'affaire?answer: Non : En vertu de l’article 44 du code des assurances, l’assurance des résidents et des risques situés en Tunisie ne peut être souscrite qu’auprès d’une entreprise d’assurances Tunisienne et ce à l’exception de l’assurance de la responsabilité civile de l’armateur et du transporteur maritime. answer: Non : En vertue de l’article 41 de la loi n°96-59 portant réforme de la monnaier, l’importation et l’ exportation de devises sont soumises au timbre à hauteur minimum de 500 dinars par lot d’effets importés ou exportés à destination de la Tunisie et ce par personne ou par entité financière ou mixte ayant une activité économique.<|endoftext|>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Choix: 60
Answer: answer: REFERENCES JURIDIQUES:
Décret n° 2000-2475 en date du 31 octobre 2000 relatif à la formalité unique pour la création des projets individuels tel que modifié par le décret n° 2008-733 du 24 mars 2008.

CONDITIONS D'OBTENTION:
Le projet doit être réalisé par une personne physique qui se charge de la gestion de façon individuelle ;
Le projet ne doit pas revêtir la forme d'une société ;
L'exercice de la profession ne doit pas nécessiter une aptitude scientifique spécifique et être soumis au contrôle du conseil ou de l'ordre professionnel concerné et ce, conformément aux règlements y afférents.
LIEU DE DEPOT ET D'OBTENTION:
A la recette des finances dont le lieu d'installation du projet relève de sa compétence.

PIECES A FOURNIR:
Un imprimé à remplir en un seul exemplaire signé par le promoteur du projet à retirer soit directement auprès des services de la recette des finances soit en ligne.
Faire accompagner la déclaration selon le cas, des pièces ci-après :
Une c

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




[Sentence]:en Tunisie,Quelle est la date limite de dépôt de la liste des factures d’achat relative aux bons de commande utilisés durant le 2ième trimestre pour une société totalement exportatrice?answer: REFERENCES JURIDIQUES:
Décret n° 2000-2475 en date du 31 octobre 2000 relatif à la formalité unique pour la création des projets individuels tel que modifié par le décret n°2004-1706 du 21 decembreet la loi de finances 2001
<|endoftext|>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Choix: 28
Answer: answer: Les opérations imposables
Le champ d’application de la TVA couvre :

Les importations sous réserve des exonérations,
La production industrielle, artisanale et les services,
Les opérations relevant des professions libérales,
Le commerce de gros autre que celui des produits alimentaires,
Le commerce de détail lorsque le chiffre d’affaires annuel global est égal ou supérieur à 100.000D à l’exclusion des produits suivants :
produits alimentaires,
Les produits soumis au régime de l’homologation administrative des prix (carburants, alcool, tabac, allumettes, cahiers scolaires, …),
Les Opérations Imposables à la TVA par option
Les entreprises qui réalisent des  opérations hors champ  d’application de la TVA ou exonérées de ladite taxe et ce au titre :

De la totalité de l’activité pour les opérations hors champ de ladite taxe ;
D’une partie de l’activité pour les produits exonérés destinés à l’exportation ou à l’approvisionnement des assujettis.
Les opérations hors c

In [53]:
gpt.eval()
with torch.no_grad():
  prompt = tokenizer("question : en tunisie quels sont les impôts ? , Exemple of answer : "+TopN(tfidf_matrix,"en tunisie quels sont les impôts ?") , truncation=True, padding=True, max_length=128, return_tensors='pt')
  prompt = {key: value.to(device) for key, value in prompt.items()}
  out = gpt.generate(**prompt, max_length=512, top_k=50, top_p=0.9, temperature=1.0, do_sample=True, repetition_penalty = 1.2, num_beams=1)
  user_input=tokenizer.decode(out[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Choix: 159
Answer: answer: L'administration peut procéder à la vérification des impôts dans la limite des délais de prescription qui suivent : 4 ans pour les impôts déclarés, 10 ans en cas de défaut de déclaration de l'impôt, 10 ans pour les droits de timbre dans tous les cas.
-----------------------
Choix: 290
Answer: answer: L'administration peut procéder à la vérification des impôts dans la limite des délais de prescription qui suivent : 4 ans pour les impôts déclarés, 10 ans en cas de défaut de déclaration de l'impôt, 10 ans pour les droits de timbre dans tous les cas.
-----------------------
Choix: 409
Answer: answer: Personnes imposables
L’IRPP est dû par toute personne physique ayant sa résidence habituelle en Tunisie. Les non-résidents sont également soumis à l’impôt sur le revenu au titre de leurs revenus de source tunisienne.

Catégories de Revenus Imposables
Les catégories de revenus imposables sont :

les bénéfices industriels et commerciaux; 
les bénéfices des professions 